In [1]:
import os
import logging
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import torch
import spacy
from transformers import AutoTokenizer, AutoModel, pipeline
from neo4j import GraphDatabase

from typing import Any, Dict
from pydantic import BaseModel, validator, Field
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_core.documents import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Function to load and preprocess data from various sources
def load_data():
    # Load CSV files
    united_dates_locations = pd.read_csv("data/united_dates_locations.csv")
    alliance_dates_locations = pd.read_csv("data/alliance_dates_locations.csv")
    air_canada_dates_locations = pd.read_csv("data/air_canada_dates_locations.csv")

    # Load text files
    with open("data/united_aircraft_details.txt", "r") as file:
        united_aircraft_details_content = file.read().split('\n\n')
    with open("data/alliance_aircraft_details.txt", "r") as file:
        alliance_aircraft_details_content = file.read().split('\n\n')
    with open("data/air_canada_aircraft_details.txt", "r") as file:
        air_canada_aircraft_details_content = file.read().split('\n\n')

    # Extract text from PDF files
    def extract_text_from_pdf(pdf_path):
        text = []
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page_num in range(len(reader.pages)):
                text.append(reader.pages[page_num].extract_text())
        return "\n".join(text)

    united_pdf_content = extract_text_from_pdf("data/united_accident_outcomes.pdf")
    alliance_pdf_content = extract_text_from_pdf("data/alliance_accident_outcomes.pdf")
    air_canada_pdf_content = extract_text_from_pdf("data/air_canada_accident_outcomes.pdf")

    # Combine data
    united_data = united_dates_locations['summary'].tolist() + united_aircraft_details_content + [united_pdf_content]
    alliance_data = alliance_dates_locations['summary'].tolist() + alliance_aircraft_details_content + [alliance_pdf_content]
    air_canada_data = air_canada_dates_locations['summary'].tolist() + air_canada_aircraft_details_content + [air_canada_pdf_content]

    return united_data, alliance_data, air_canada_data

# Load the data
united_data, alliance_data, air_canada_data = load_data()

In [3]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load models
nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
rel_extractor = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Load Hugging Face local embeddings
hugging_face_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Initialize document stores
class InMemoryDocstore:
    def __init__(self, documents):
        self.documents = documents

    def get_document(self, doc_id):
        return self.documents.get(doc_id, None)

In [5]:
def vectorize_and_index(data_list, index_path):
    embeddings = np.vstack([embed_text(text) for text in data_list])
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings.astype('float32'))
    faiss.write_index(index, index_path)

vectorize_and_index(united_data, "faiss_indexes/united_faiss.index")
vectorize_and_index(alliance_data, "faiss_indexes/alliance_faiss.index")
vectorize_and_index(air_canada_data, "faiss_indexes/air_canada_faiss.index")

# Initialize document stores
united_docstore = InMemoryDocstore({str(i): Document(page_content=text, metadata={"doc_id": f"united_{i}"}) for i, text in enumerate(united_data)})
alliance_docstore = InMemoryDocstore({str(i): Document(page_content=text, metadata={"doc_id": f"alliance_{i}"}) for i, text in enumerate(alliance_data)})
air_canada_docstore = InMemoryDocstore({str(i): Document(page_content=text, metadata={"doc_id": f"air_canada_{i}"}) for i, text in enumerate(air_canada_data)})

def create_faiss_retriever(index_path, docstore, data_list):
    index = faiss.read_index(index_path)
    index_to_docstore_id = {i: str(i) for i in range(len(data_list))}
    return FAISS(embedding_function=embed_text, index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

united_faiss_retriever = create_faiss_retriever("faiss_indexes/united_faiss.index", united_docstore, united_data)
alliance_faiss_retriever = create_faiss_retriever("faiss_indexes/alliance_faiss.index", alliance_docstore, alliance_data)
air_canada_faiss_retriever = create_faiss_retriever("faiss_indexes/air_canada_faiss.index", air_canada_docstore, air_canada_data)

In [6]:
class GraphDBClient:
    def __init__(self, uri, user, password):
        self.uri = uri
        self.username = user
        self.password = password
        self.driver = None
        self.connect()

    def connect(self):
        try:
            self.driver = GraphDatabase.driver(self.uri, auth=(self.username, self.password))
            logging.info("Successfully connected to Neo4j.")
        except Exception as e:
            logging.error(f"Failed to connect to Neo4j: {e}")
            raise ConnectionError(f"Failed to connect to Neo4j: {e}")

    def close(self):
        if self.driver:
            self.driver.close()

In [7]:
# Initialize the Ollama gemma:2b model
generative_model = Ollama(model="gemma:2b")

# Define a detailed prompt template for extracting entities
entity_prompt = PromptTemplate(template="""
Extract all relevant entities from the provided text and classify them into categories.
Text: {text}
Entities:
""")

relationship_prompt = PromptTemplate(template="""
Identify the relationship between the following entities based on the provided text.
Text: {text}
Entity 1: {entity1}
Entity 2: {entity2}
Relationship:
""")

# Define LLMChain for entities and relationships
entity_chain = LLMChain(llm=generative_model, prompt=entity_prompt)
relationship_chain = LLMChain(llm=generative_model, prompt=relationship_prompt)

def extract_entities_with_llm(text):
    response = entity_chain.run({"text": text})
    logging.info(f"Entity extraction response: {response}")
    entities = []
    for line in response.split("\n"):
        if line.strip():
            parts = line.split(":")
            if len(parts) == 2:
                entity_text, entity_label = parts[0].strip(), parts[1].strip()
                entities.append((entity_text, entity_label))
    return entities

def extract_relationships_with_llm(text, entities):
    relationships = []
    for i in range(len(entities) - 1):
        for j in range(i + 1, len(entities)):
            entity1, entity2 = entities[i][0], entities[j][0]
            response = relationship_chain.run({"text": text, "entity1": entity1, "entity2": entity2})
            logging.info(f"Relationship extraction response for {entity1} and {entity2}: {response}")
            relationship = response.strip()
            relationships.append((entity1, relationship, entity2))
    return relationships

def extract_entities_and_relationships(text):
    entities = extract_entities_with_llm(text)
    relationships = extract_relationships_with_llm(text, entities)
    return entities, relationships

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
def create_knowledge_graph(conn, datasets):
    try:
        with conn.driver.session() as session:
            for dataset_name, data in datasets.items():
                for entry in data:
                    text_data = entry  # Using the text entry as the input text
                    
                    # Extract entities and relationships
                    entities, relationships = extract_entities_and_relationships(text_data)
                    
                    # Log extracted entities and relationships
                    logging.info(f"Extracted entities: {entities}")
                    logging.info(f"Extracted relationships: {relationships}")
                    
                    # Create Dataset node
                    session.run("""
                    MERGE (d:Dataset {name: $dataset_name})
                    RETURN d
                    """, parameters={'dataset_name': dataset_name})
                    logging.info(f"Dataset node created for {dataset_name}")
                    
                    # Create Entry node
                    session.run("""
                    MERGE (e:Entry {text: $text_data})
                    RETURN e
                    """, parameters={'text_data': text_data})
                    logging.info(f"Entry node created for {text_data}")
                    
                    # Create relationships between Dataset and Entry
                    session.run("""
                    MATCH (d:Dataset {name: $dataset_name})
                    MATCH (e:Entry {text: $text_data})
                    MERGE (d)-[:CONTAINS_ENTRY]->(e)
                    """, parameters={'dataset_name': dataset_name, 'text_data': text_data})
                    logging.info(f"Relationship created between Dataset {dataset_name} and Entry {text_data}")
                    
                    # Create entities and relationships to the Entry
                    for entity_text, entity_label in entities:
                        session.run("""
                        MERGE (en:Entity {name: $entity, type: $type})
                        MERGE (e:Entry {text: $text_data})-[:MENTIONS]->(en)
                        """, parameters={'entity': entity_text, 'type': entity_label, 'text_data': text_data})
                        logging.info(f"Entity node created for {entity_text} with label {entity_label}, related to Entry {text_data}")
                    
                    # Create relationships between entities based on extracted relationships
                    for entity1, relation, entity2 in relationships:
                        session.run("""
                        MATCH (en1:Entity {name: $entity1}), (en2:Entity {name: $entity2})
                        MERGE (en1)-[r:RELATION {type: $relation, source: $source}]->(en2)
                        """, parameters={'entity1': entity1, 'entity2': entity2, 'relation': relation, 'source': dataset_name})
                        logging.info(f"Relationship {relation} created between Entity {entity1} and Entity {entity2} for Dataset {dataset_name}")
    except Exception as e:
        logging.error(f"Error creating knowledge graph: {e}")

# Initialize connection to Neo4j
graphdb_client = GraphDBClient(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), password=os.getenv('NEO4J_PASSWORD'))

# Combine all datasets
datasets = {
    "United_Airlines": united_data,
    "Alliance_Airlines": alliance_data,
    "Air_Canada": air_canada_data
}
create_knowledge_graph(graphdb_client, datasets)

INFO:root:Successfully connected to Neo4j.
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:root:Entity extraction response: **Categories:**

**Events:**

* Uncontained engine failure

**Subjects:**

* Engine

**Objects:**

* Hydraulic systems

**Relationships:**

* Uncontained engine failure led to loss of all hydraulic systems
INFO:root:Relationship extraction response for **Categories and **Events: Sure. Based on the context, the relationship between the entities is:

**Entity 1: Categories**

**Entity 2: Events**

**Relationship:** The **Categories** entity contains the **Events** entity as a subset. This means that the **Categories** entity provides a more general classification for the **Events** entity.

In other words, the **Categories** entity encompasses the **E

In [20]:
graphdb_client = GraphDBClient(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), password=os.getenv('NEO4J_PASSWORD'))

def query_knowledge_graph(client, keyword):
    query = """
    MATCH (e:Entity)-[r:RELATION]->(related:Entity)
    WHERE toLower(e.name) CONTAINS toLower($keyword) 
       OR toLower(e.type) CONTAINS toLower($keyword)
       OR toLower(related.name) CONTAINS toLower($keyword)
       OR toLower(related.type) CONTAINS toLower($keyword)
       OR toLower(type(r)) CONTAINS toLower($keyword)
    RETURN e.name AS entity, e.type AS entity_type, type(r) AS relationship, related.name AS related_entity, related.type AS related_entity_type
    """
    logging.info(f"Running query with keyword: {keyword}")
    try:
        with client.driver.session() as session:
            result = session.run(query, keyword=keyword)
            records = [record.data() for record in result]
            if not records:
                logging.info(f"No results found for keyword: {keyword}")
            else:
                logging.info(f"Found {len(records)} records for keyword: {keyword}")
            return records
    except Exception as e:
        logging.error(f"Error querying knowledge graph: {e}")
        return []

# Example query
keyword = "Boeing"
results = query_knowledge_graph(graphdb_client, keyword)
for record in results:
    print(f"Entity: {record['entity']} ({record['entity_type']}), Relationship: {record['relationship']}, Related Entity: {record['related_entity']} ({record['related_entity_type']})")


INFO:root:Successfully connected to Neo4j.
INFO:root:Running query with keyword: Boeing
INFO:root:Found 48 records for keyword: Boeing


Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 2 (People**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 2 (People**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 3 (Locations**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 3 (Locations**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 4 (Organizations**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 2 (Aircraft Details**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 2 (Aircraft Details**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 3 (Other Relevant Information**)
Entity: * Aircraft Type (Boeing 777-200), Relationship: RELATION, Related Entity: **Category 3 (Oth

In [10]:
class CustomMultiRetriever(BaseModel):
    faiss_retrievers: Dict[str, FAISS] = Field(..., description="Dictionary of FAISS retrievers")
    knowledge_graph_client: Any

    @validator('faiss_retrievers')
    def check_faiss_retrievers(cls, value):
        if not value:
            raise ValueError("FAISS retrievers cannot be empty")
        return value

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query):
        query_vector = embed_text(query).reshape(1, -1)
        results = {}
        for airline, retriever in self.faiss_retrievers.items():
            distances, indices = retriever.index.search(query_vector, k=10)
            docs = [retriever.docstore.get_document(str(i)) for i in indices[0] if retriever.docstore.get_document(str(i)) is not None]
            results[airline] = docs
        return results


/var/folders/8_/w8p7c86x5_v0716y6rvk1sf40000gn/T/ipykernel_98747/726269644.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('faiss_retrievers')


In [11]:
def integrate_graph_and_docs(graph_results, retrieved_docs):
    integrated_context = "Retrieved Context:\n"
    if graph_results:
        for result in graph_results:
            entity = result.get('entity', 'Unknown Entity')
            relationship = result.get('relationship', 'Unknown Relationship')
            related_entity = result.get('related_entity', 'Unknown Related Entity')
            integrated_context += f"{entity} {relationship} {related_entity}. "
    else:
        integrated_context += "No relevant data found in the knowledge graph.\n"
    
    if retrieved_docs:
        integrated_context += "\nFrom FAISS:\n"
        for doc in retrieved_docs:
            integrated_context += doc.page_content + "\n"
    else:
        integrated_context += "No relevant data found in the FAISS retrievers.\n"
    
    return integrated_context

In [12]:
def extract_entities(query):
    doc = nlp(query)
    keywords = [ent.text for ent in doc.ents if ent.label_ in ("ORG", "GPE", "NORP", "PRODUCT", "EVENT")]
    return keywords

In [13]:
def process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain):
    try:
        entities = extract_entities(user_query)
        logger.info(f"Extracted entities: {entities}")

        graph_context = ""
        for entity in entities:
            graph_results = query_knowledge_graph(graphdb_client, entity)
            logger.info(f"Graph results for entity {entity}: {graph_results}")
            if graph_results:
                graph_context += integrate_graph_and_docs(graph_results, [])
            else:
                graph_context += f"No relevant data found in the knowledge graph for entity: {entity}.\n"

        faiss_docs = custom_multi_retriever.get_relevant_documents(user_query)

        final_context = graph_context + " ".join([doc.page_content for docs in faiss_docs.values() for doc in docs])

        response = llm_chain({"context": final_context})

        # Extract doc_id from retrieved documents
        retrieved_doc_ids = {key: [doc.metadata["doc_id"] for doc in docs] for key, docs in faiss_docs.items()}
        return response['text'], retrieved_doc_ids
    except Exception as e:
        logger.error(f"Error processing query: {e}")
        raise



In [21]:
# Initialize the generative model
generative_model = Ollama(model="gemma:7b")

# Define a prompt template
prompt_template = PromptTemplate(
    template="""
    You are an AI assistant that specializes in providing detailed information about airline accidents. 
    When given a query about a specific flight, you should:

    1. Identify the flight number and any other relevant details from the query.
    2. Retrieve specific information about the flight from the provided context, including any relevant accidents or incidents.
    3. Summarize the information in a clear and concise manner within one sentence.
    4. If there are multiple incidents related to the flight, provide details on each incident separately.
    5. Ensure the response is focused on the specific flight mentioned in the query.

    Use the provided context to generate the response and avoid including unrelated information.

    Context:
    {context}
    """,
    input_variables=["context"]
)

# Initialize connection to Neo4j
graphdb_client = GraphDBClient(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), password=os.getenv('NEO4J_PASSWORD'))

custom_multi_retriever = CustomMultiRetriever(
    faiss_retrievers={
        "United Airlines": united_faiss_retriever,
        "Alliance Airlines": alliance_faiss_retriever,
        "Air Canada": air_canada_faiss_retriever
    },
    knowledge_graph_client=graphdb_client
)

# Create the LLMChain
llm_chain = LLMChain(prompt=prompt_template, llm=generative_model)

user_query = "Tell me about what aircraft FLIGHT 624 faced accident"
response,sample_doc_ret = process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain)
print(f"Response: {response}")


INFO:root:Successfully connected to Neo4j.
INFO:__main__:Extracted entities: ['FLIGHT 624']
INFO:root:Running query with keyword: FLIGHT 624
INFO:root:Found 31 records for keyword: FLIGHT 624
INFO:__main__:Graph results for entity FLIGHT 624: [{'entity': '- Air Canada Flight 624 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Air Canada Flight 624 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Locations**'}, {'entity': '- Air Canada Flight 624 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'Aircraft Details**'}, {'entity': '- Air Canada Flight 624 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Other Relevant Information**'}, {'entity': '- Air Canada Flight 624 (Accide

Response: **Summary of Airline Accident Records**

The provided airline accident records showcase the diverse range of aviation incidents that have occurred, highlighting the importance of safety protocols, training, and infrastructure in preventing accidents.

**Key Highlights:**

* **Accident 1: Alliance Airlines Flight 764:** This incident revealed a hydraulic system failure, leading to an emergency landing and subsequent updates to maintenance protocols to prevent similar occurrences.
* **Accident 2: Air Canada Flight 624:** This event involved a hard landing due to power line strikes and subsequent passenger deaths, prompting improvements in landing procedures and safety measures.
* **Accident 3: Alliance Airlines Flight 982:** This incident demonstrated the impact of turbulence on passenger safety, highlighting the importance of seatbelt use and turbulence response training.

**Conclusion:**

The airline accident records provide valuable insights into the risks and safety measure

In [15]:
print(sample_doc_ret)

{'United Airlines': ['united_6', 'united_5', 'united_4', 'united_7', 'united_3', 'united_2', 'united_1', 'united_0'], 'Alliance Airlines': ['alliance_4', 'alliance_5', 'alliance_6', 'alliance_7', 'alliance_3', 'alliance_0', 'alliance_1', 'alliance_2'], 'Air Canada': ['air_canada_5', 'air_canada_4', 'air_canada_6', 'air_canada_7', 'air_canada_3', 'air_canada_2', 'air_canada_0', 'air_canada_1']}


In [16]:
#user_query = "Tell me about what aircraft FLIGHT 328"
#response = process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain)
#print(f"Response: {response}")

In [17]:
from sklearn.metrics import precision_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

def precision_at_k(retrieved_docs, relevant_docs, k=10):
    relevant_set = set(relevant_docs)
    retrieved_k = retrieved_docs[:k]
    retrieved_set = set(retrieved_k)
    intersection = retrieved_set.intersection(relevant_set)
    precision = len(intersection) / len(retrieved_k) if retrieved_k else 0
    return precision

def mean_reciprocal_rank(retrieved_docs, relevant_docs):
    for i, doc in enumerate(retrieved_docs):
        if doc in relevant_docs:
            return 1 / (i + 1)
    return 0

def ndcg_at_k(retrieved_docs, relevant_docs, k=10):
    dcg = 0
    for i, doc in enumerate(retrieved_docs[:k]):
        if doc in relevant_docs:
            dcg += 1 / np.log2(i + 2)
    idcg = sum(1 / np.log2(i + 2) for i in range(min(len(relevant_docs), k)))
    return dcg / idcg if idcg > 0 else 0

def bleu_score(reference, hypothesis):
    smoothing_function = SmoothingFunction().method1
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()
    return sentence_bleu([reference_tokens], hypothesis_tokens, smoothing_function=smoothing_function)

# Function to evaluate multiple metrics
def evaluate_performance(retrieved_docs, relevant_docs, generated_response, reference_response, k=10):
    precision = precision_at_k(retrieved_docs, relevant_docs, k)
    mrr = mean_reciprocal_rank(retrieved_docs, relevant_docs)
    ndcg = ndcg_at_k(retrieved_docs, relevant_docs, k)
    bleu = bleu_score(reference_response, generated_response)
    return precision, mrr, ndcg, bleu


In [18]:
test_queries = [
    {
        "query": "Tell me about United Airlines Flight 232",
        "relevant_docs": ["united_0","united_1","united_2"],
        "expected_response": "United Airlines Flight 232 suffered an uncontained engine failure, leading to the loss of all hydraulic systems. The aircraft broke apart upon landing. Out of 296 passengers and crew, 111 were killed, and 185 survived."
    },
    {
        "query": "Details of United Airlines Flight 93",
        "relevant_docs": ["united_3","united_4","united_5"],
        "expected_response": "United Airlines Flight 93 was hijacked during the September 11 attacks. The plane crashed into a field, killing all 44 people on board. The passengers' actions prevented the aircraft from reaching its intended target."
    },
    {
        "query": "Incident with United Airlines Flight 328",
        "relevant_docs": ["united_6","united_7","united_8"],
        "expected_response": "United Airlines Flight 328 experienced a right engine failure shortly after takeoff. The aircraft returned to Denver International Airport for an emergency landing with no injuries."
    },
    {
        "query": "Tell me about Air Canada Flight 797",
        "relevant_docs": ["air_canada_0","air_canada_1","air_canada_5"],
        "expected_response": "Air Canada Flight 797 had a fire start in the lavatory while in flight. The aircraft landed, but the fire rapidly spread, resulting in the deaths of 23 passengers. 21 passengers and all crew members survived."
    },
    {
        "query": "Details of Air Canada Flight 624",
        "relevant_docs": ["air_canada_0","air_canada_1","air_canada_2"],
        "expected_response": "Air Canada Flight 624 struck power lines and an antenna array upon approach in heavy snow at Halifax Stanfield International Airport. All passengers and crew evacuated safely, with 25 people sustaining minor injuries."
    },
    {
        "query": "Incident with Air Canada Flight 759",
        "relevant_docs": ["air_canada_3","air_canada_4","air_canada_5"],
        "expected_response": "Air Canada Flight 759 was cleared to land on a runway that was closed for maintenance at San Francisco International Airport. The aircraft almost landed on a taxiway, but the incident was averted, and it landed safely on a different runway."
    },
    {
        "query": "Tell me about Alliance Airlines Flight 764",
        "relevant_docs": ["alliance_0","alliance_1","alliance_2"],
        "expected_response": "Alliance Airlines Flight 764 suffered a hydraulic system failure shortly after takeoff. The aircraft returned to Brisbane and landed safely without any injuries."
    },
    {
        "query": "Details of Alliance Airlines Flight 850",
        "relevant_docs": ["alliance_3","alliance_4","alliance_5"],
        "expected_response": "Alliance Airlines Flight 850 struck a flock of birds during final approach to Perth Airport, causing significant damage to the right engine. The aircraft landed safely with no injuries."
    },
    {
        "query": "Incident with Alliance Airlines Flight 982",
        "relevant_docs": ["alliance_6","alliance_7","alliance_8"],
        "expected_response": "Alliance Airlines Flight 982 encountered severe turbulence while descending towards Melbourne. Several passengers were injured, but the aircraft landed safely."
    }
]

results = []

# Initialize connection to Neo4j
graphdb_client = GraphDBClient(uri=os.getenv('NEO4J_URI'), user=os.getenv('NEO4J_USERNAME'), password=os.getenv('NEO4J_PASSWORD'))

# Evaluate Performance
for test_case in test_queries:
    user_query = test_case["query"]
    relevant_docs = test_case["relevant_docs"]
    reference_response = test_case["expected_response"]

    generated_response, retrieved_docs = process_query(user_query, custom_multi_retriever, graphdb_client, llm_chain)
    print(retrieved_docs)

    # Flatten the retrieved documents
    retrieved_doc_ids = [doc_id for doc_list in retrieved_docs.values() for doc_id in doc_list]

    precision, mrr, ndcg, bleu = evaluate_performance(retrieved_doc_ids, relevant_docs, generated_response, reference_response)
    
    results.append({
        "query": user_query,
        "precision@10": precision,
        "mrr": mrr,
        "ndcg@10": ndcg,
        "bleu": bleu,
        "generated_response": generated_response
    })


graphdb_client.close()

INFO:root:Successfully connected to Neo4j.
INFO:__main__:Extracted entities: ['United Airlines', 'Flight 232']
INFO:root:Running query with keyword: United Airlines
INFO:root:Found 45 records for keyword: United Airlines
INFO:__main__:Graph results for entity United Airlines: [{'entity': '- United Airlines Flight 232 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- United Airlines Flight 93 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- United Airlines Flight 328 (Accident ID', 'entity_type': '3)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- United Airlines Flight 232 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Locations**'}, {'entity': '- United

{'United Airlines': ['united_4', 'united_7', 'united_5', 'united_6', 'united_3', 'united_1', 'united_0', 'united_2'], 'Alliance Airlines': ['alliance_7', 'alliance_6', 'alliance_5', 'alliance_4', 'alliance_3', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_4', 'air_canada_5', 'air_canada_7', 'air_canada_2', 'air_canada_3', 'air_canada_0', 'air_canada_1']}


INFO:root:Found 16 records for keyword: Flight 93
INFO:__main__:Graph results for entity Flight 93: [{'entity': '- United Airlines Flight 93 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- United Airlines Flight 93 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Locations**'}, {'entity': '- United Airlines Flight 93 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'Aircraft Details**'}, {'entity': '- United Airlines Flight 93 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Other Relevant Information**'}, {'entity': '- United Airlines Flight 93 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'Aircraft

{'United Airlines': ['united_5', 'united_4', 'united_7', 'united_3', 'united_6', 'united_1', 'united_0', 'united_2'], 'Alliance Airlines': ['alliance_6', 'alliance_7', 'alliance_4', 'alliance_5', 'alliance_3', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_4', 'air_canada_7', 'air_canada_5', 'air_canada_3', 'air_canada_2', 'air_canada_0', 'air_canada_1']}


INFO:__main__:Extracted entities: ['Air Canada']
INFO:root:Running query with keyword: Air Canada
INFO:root:Found 90 records for keyword: Air Canada
INFO:__main__:Graph results for entity Air Canada: [{'entity': '- Air Canada Flight 797 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Air Canada Flight 624 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Air Canada Flight 759 (Accident ID', 'entity_type': '3)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Air Canada Flight 797 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Locations**'}, {'entity': '- Air Canada Flight 624 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity'

...

{'United Airlines': ['united_5', 'united_4', 'united_7', 'united_6', 'united_1', 'united_3', 'united_2', 'united_0'], 'Alliance Airlines': ['alliance_6', 'alliance_4', 'alliance_5', 'alliance_7', 'alliance_3', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_4', 'air_canada_7', 'air_canada_5', 'air_canada_3', 'air_canada_2', 'air_canada_0', 'air_canada_1']}


INFO:__main__:Extracted entities: ['Alliance Airlines']
INFO:root:Running query with keyword: Alliance Airlines
INFO:root:Found 78 records for keyword: Alliance Airlines
INFO:__main__:Graph results for entity Alliance Airlines: [{'entity': '- Alliance Airlines Flight 764 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Alliance Airlines Flight 850 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Alliance Airlines Flight 982 (Accident ID', 'entity_type': '3)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Alliance Airlines Flight 764 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Locations**'}, {'entity': '- Alliance Airlines Flight 850 (Accident ID', 'e

{'United Airlines': ['united_5', 'united_6', 'united_7', 'united_4', 'united_3', 'united_1', 'united_2', 'united_0'], 'Alliance Airlines': ['alliance_4', 'alliance_6', 'alliance_5', 'alliance_7', 'alliance_3', 'alliance_2', 'alliance_0', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_5', 'air_canada_7', 'air_canada_4', 'air_canada_2', 'air_canada_3', 'air_canada_0', 'air_canada_1']}


INFO:__main__:Extracted entities: ['Alliance Airlines', 'Flight 982']
INFO:root:Running query with keyword: Alliance Airlines
INFO:root:Found 78 records for keyword: Alliance Airlines
INFO:__main__:Graph results for entity Alliance Airlines: [{'entity': '- Alliance Airlines Flight 764 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Alliance Airlines Flight 850 (Accident ID', 'entity_type': '2)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Alliance Airlines Flight 982 (Accident ID', 'entity_type': '3)', 'relationship': 'RELATION', 'related_entity': '**Category 2', 'related_entity_type': 'People**'}, {'entity': '- Alliance Airlines Flight 764 (Accident ID', 'entity_type': '1)', 'relationship': 'RELATION', 'related_entity': '**Category 3', 'related_entity_type': 'Locations**'}, {'entity': '- Alliance Airlines Flight 850 (Ac

{'United Airlines': ['united_6', 'united_5', 'united_7', 'united_4', 'united_3', 'united_1', 'united_2', 'united_0'], 'Alliance Airlines': ['alliance_5', 'alliance_3', 'alliance_6', 'alliance_7', 'alliance_4', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_5', 'air_canada_4', 'air_canada_7', 'air_canada_3', 'air_canada_2', 'air_canada_0', 'air_canada_1']}
{'United Airlines': ['united_5', 'united_6', 'united_4', 'united_7', 'united_3', 'united_1', 'united_2', 'united_0'], 'Alliance Airlines': ['alliance_6', 'alliance_5', 'alliance_4', 'alliance_7', 'alliance_3', 'alliance_0', 'alliance_2', 'alliance_1'], 'Air Canada': ['air_canada_6', 'air_canada_4', 'air_canada_7', 'air_canada_5', 'air_canada_2', 'air_canada_3', 'air_canada_0', 'air_canada_1']}


In [19]:
# Print Evaluation Results
for result in results:
    print(f"Query: {result['query']}")
    print(f"Precision@10: {result['precision@10']}")
    print(f"MRR: {result['mrr']}")
    print(f"nDCG@10: {result['ndcg@10']}")
    print(f"BLEU: {result['bleu']}")

Query: Tell me about United Airlines Flight 232
Precision@10: 0.3
MRR: 0.16666666666666666
nDCG@10: 0.4716276524567204
BLEU: 0.006774301998022002
Query: Details of United Airlines Flight 93
Precision@10: 0.3
MRR: 1.0
nDCG@10: 0.9674679834891693
BLEU: 0.08792722108099439
Query: Incident with United Airlines Flight 328
Precision@10: 0.2
MRR: 1.0
nDCG@10: 0.6713860725233041
BLEU: 0.07014903299135951
Query: Tell me about Air Canada Flight 797
Precision@10: 0.0
MRR: 0.05263157894736842
nDCG@10: 0.0
BLEU: 0.012754457182271348
Query: Details of Air Canada Flight 624
Precision@10: 0.0
MRR: 0.045454545454545456
nDCG@10: 0.0
BLEU: 0.017466590550973486
Query: Incident with Air Canada Flight 759
Precision@10: 0.0
MRR: 0.05555555555555555
nDCG@10: 0.0
BLEU: 0.004293705377190336
Query: Tell me about Alliance Airlines Flight 764
Precision@10: 0.0
MRR: 0.07142857142857142
nDCG@10: 0.0
BLEU: 0.003511151209367814
Query: Details of Alliance Airlines Flight 850
Precision@10: 0.2
MRR: 0.1111111111111111
nD